[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ar-yansingh/Capstone-Project/blob/main/Model1_Final.ipynb)

In [15]:

!pip install pathway



In [74]:

import pandas as pd
import pathway as pw
from pathway.internals import this
import time
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

In [58]:

# LOAD DATA WITH PANDAS

df = pd.read_csv("dataset.csv")


In [59]:

# DEFINE PATHWAY SCHEMA

class ParkingData(pw.Schema):
    LastUpdatedDate: str
    LastUpdatedTime: str
    Capacity: float
    Occupancy: float
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: float
    IsSpecialDay: int

In [60]:

# LOAD INTO PATHWAY

table = pw.debug.table_from_pandas(df)


**Compute Model 1 Pricing Logic**

In [84]:

# Combine date and time into a timestamp
table = table.with_columns(
    Timestamp = pw.apply(lambda date, time: f"{date} {time}", table.LastUpdatedDate, table.LastUpdatedTime)
)

# Compute occupancy ratio
table = table.with_columns(
    OccRatio = pw.if_else(table.Capacity > 0, table.Occupancy / table.Capacity, 0)
)

# Raw price calculation (linear model)
table = table.with_columns(
    RawPrice = 10 + 2.0 * table.OccRatio
)

# Bound price between $5 and $20
table = table.with_columns(
    Price = pw.if_else(table.RawPrice < 5, 5,
             pw.if_else(table.RawPrice > 20, 20, table.RawPrice))
)


In [68]:
# RUN THE PIPELINE

start_time = time.time()
pw.run()
print(f"✅ Pathway pipeline executed in {time.time() - start_time:.2f} seconds.")

Output()

✅ Pathway pipeline executed in 0.02 seconds.


In [63]:

# EXPORT TO PANDAS
start_time = time.time()
df_prices = pw.debug.table_to_pandas(
    table.select(
        this.Timestamp,
        this.Price,
        this.VehicleType,
        this.QueueLength,
        this.TrafficConditionNearby,
        this.IsSpecialDay
    )
)

In [64]:
# Show final table
df_prices.head()

,Timestamp,Price,VehicleType,QueueLength,TrafficConditionNearby,IsSpecialDay
^RT08X9R6J1BVNPFSV7WM9SG100,11-10-2016 14:04:00,11.772610,car,4,average,0
^E6E8PYV0E39J85WG6TYB5B0800,26-10-2016 07:59:00,11.068041,car,2,average,0
^M8M1652S9R57Z5GDH3DHGBGD00,12-11-2016 09:00:00,10.226667,car,2,low,0
^49HDW7592649GY7S9P2W6FGE00,20-11-2016 08:27:00,10.339943,car,3,low,1
^8WN10YPX5631SHRF0PSGHCRE00,24-11-2016 11:00:00,11.015129,car,6,average,0


In [69]:
# Convert Timestamp to datetime

df_prices['Timestamp'] = pd.to_datetime(df_prices['Timestamp'], format="%d-%m-%Y %H:%M:%S")

# Sort and reset index

df_prices = df_prices.sort_values(by='Timestamp').reset_index(drop=True)

# Ensure correct dtypes for all fields used in tooltips

df_prices['Price'] = df_prices['Price'].astype(float)
df_prices['QueueLength'] = df_prices['QueueLength'].astype(float)
df_prices['VehicleType'] = df_prices['VehicleType'].astype(str)
df_prices['TrafficConditionNearby'] = df_prices['TrafficConditionNearby'].astype(str)
df_prices['IsSpecialDay'] = df_prices['IsSpecialDay'].astype(str)


In [81]:
# Make Bokeh output appear directly in the notebook
output_notebook()

# Prepare the data source for Bokeh from your DataFrame
source = ColumnDataSource(df_prices)

# Define what information will appear in the hover tooltip for each point
tooltips = [
    ("Time", "@Timestamp{%F %H:%M}"),
    ("Price", "@Price{0.2f}"),
    ("Vehicle", "@VehicleType"),
    ("Queue", "@QueueLength{0}"),
    ("Traffic", "@TrafficConditionNearby"),
    ("Special Day", "@IsSpecialDay"),
]

# Create the figure for your time series plot
p = figure(
    x_axis_type='datetime',
    title='Dynamic Parking Price Over Time (Model 1)',
    x_axis_label='Time',
    y_axis_label='Price ($)',
    width=950,
    height=450,
    tools="pan,wheel_zoom,box_zoom,reset"
)

# Draw the main green line connecting all price points over time
# Thin line width create the "spike" effect seen in dense time series
p.line('Timestamp', 'Price', source=source, line_width=1, color='green', legend_label="Model 1 Price")

# Add small black circles at each data point for extra clarity
p.circle('Timestamp', 'Price', source=source, size=3, color='black', alpha=0.7)

# Set up the hover tool so users can see details when hovering over the plot
hover = HoverTool(
    tooltips=tooltips,
    formatters={'@Timestamp': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

# Place the legend in the top left corner for easy reference
p.legend.location = 'top_left'

# Make the grid lines slightly transparent for a cleaner look
p.grid.grid_line_alpha = 0.3


show(p)